<a href="https://colab.research.google.com/github/veren4/SMILES_featurization/blob/master/LSTM_KDNuggets_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook ist based on [this tutorial](https://www.kdnuggets.com/2020/07/pytorch-lstm-text-generation-tutorial.html). The code is from their Github repo.

Open problems with this model:
* Wenn ich in dem Satz, den ich hinten reinfüttere zum Predicten, ein Zeichen habe, das im Trainings-Datensatz nicht vorkam, kriege ich einen Fehler. => Generell muss ich unknown tokens einführen.
* Ich schaue das Vokabular des ganzen Datensatzes an. Wenn ich den aber am Anfang nicht einlese, geht das nicht => Vorher bestimmen und hier nur einlesen!
* Datensatz einlesen, ohne komplett in den Cache zu laden ([Massive Dataset class](https://github.com/pytorch/text/issues/130))
* Adapt lstm size and embedding dim?

#Setup RUN

In [1]:
import torch
import pandas as pd
from collections import Counter
from torch import nn, optim
import argparse
import numpy as np
from torch.utils.data import DataLoader

!pip install -q SmilesPE
from SmilesPE.pretokenizer import atomwise_tokenizer
import pickle
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import platform
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('Python: ', platform.python_version())
print('PyTorch: ', torch.__version__)
if(device.type == 'cuda'):
  print('Using GPU (cuda)')
else:
  print('Using CPU!')

Python:  3.6.9
PyTorch:  1.7.0+cu101
Using GPU (cuda)


#Metdata RUN

* Number of lines
* regex: cut away line number + tabs/spaces
* byte offset 

wc -l CID-SMILES\
108826964 CID-SMILES

108 826 964   lines are in that file.

In [4]:
'''
data_stream = open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/CID-SMILES-first100', 'r')
#data_stream = open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/CID-SMILES', 'r')
#data_stream = open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/worded_smiles_no_header.csv', 'r')

offset_list = list()

number_of_lines = 100
#number_of_lines = 108826964
#number_of_lines = 14

for i in range(number_of_lines):
  data_stream.readline()
  offset_list.append(data_stream.tell())

data_stream.close()

with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list_CID-first100.pkl', 'wb') as fid:
#with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list.pkl', 'wb') as fid:
#with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list_worded_smiles_no_header.pkl', 'wb') as fid:

  pickle.dump(offset_list, fid)
'''

"\ndata_stream = open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/CID-SMILES-first100', 'r')\n#data_stream = open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/CID-SMILES', 'r')\n#data_stream = open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/worded_smiles_no_header.csv', 'r')\n\noffset_list = list()\n\nnumber_of_lines = 100\n#number_of_lines = 108826964\n#number_of_lines = 14\n\nfor i in range(number_of_lines):\n  data_stream.readline()\n  offset_list.append(data_stream.tell())\n\ndata_stream.close()\n\nwith open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list_CID-first100.pkl', 'wb') as fid:\n#with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list.pkl', 'wb') as fid:\n#with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list_worded_smiles_no_header.pkl', 'wb') as fid:\n\n  pi

In [5]:
with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list_CID-first100.pkl', 'rb') as fid:
     offset_list = pickle.load(fid)

#with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list.pkl', 'rb') as fid:
#     offset_list = pickle.load(fid)

#with open('/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/offset_list_worded_smiles_no_header.pkl', 'rb') as fid:
#     offset_list = pickle.load(fid)

In [6]:
dict_offset = { i : offset_list[i] for i in range(0, len(offset_list) ) }

#Dataset

##Standard Dataset class

In [19]:
#import torch
#import pandas as pd


class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        args,
    ):
        self.args = args
        self.words = self.load_words()
        self.uniq_words = ['UNK', 'SOL', 'EOL', 'PAD', '1', 'N', ')', 'C', 'S', '=', '4', 'O', '(', '2', '3', 'P']

        # tokenization dictionaries (numerization)
        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        # numericalize all the tokens
        #self.words_indexes = [self.word_to_index[w] for w in self.words]
        # if the token is in word_to index, then the index, otherwise 'UNK' = 0
        # [f(x) if condition else g(x) for x in sequence]
        #self.words_indexes = [self.word_to_index[w] if (w in uniq_words) else self.word_to_index['UNK'] for w in self.words]
        self.words_indexes = [self.word_to_index[w] if (w in self.uniq_words) else 0 for w in self.words]

    def load_words(self):

        #train_df = pd.read_csv('data/worded_smiles.csv')
        infile = '/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/worded_smiles.csv'
        with open(infile, "r") as file1:
            train_df = pd.read_csv(file1)
        file1.close()

        # Tokenize
        train_df['tokenized_SMILES'] = ''
        for row in range(train_df.shape[0]):
          train_df.loc[row, 'tokenized_SMILES'] = atomwise_tokenizer(train_df.loc[row, 'SMILES'])
        
        # Padding + SOL + EOL
        for row in range(train_df.shape[0]):              # ATTENTION: Are the column indexes correct?!
          actual_length = len(train_df.loc[row, 'tokenized_SMILES'])
          length_before_delimiters = self.args.sequence_length - 2

          if actual_length > length_before_delimiters:
            train_df.loc[row, 'tokenized_SMILES'] = train_df.loc[row, 'tokenized_SMILES'][:length_before_delimiters]
            train_df.loc[row, 'tokenized_SMILES'].append('EOL')
          elif actual_length < length_before_delimiters:
            temp = ['UNK']*length_before_delimiters
            shortie = train_df.loc[row, 'tokenized_SMILES']
            shortie.append('EOL')
            temp[:actual_length] = shortie
            train_df.loc[row, 'tokenized_SMILES'] = temp
          train_df.loc[row, 'tokenized_SMILES'].insert(0, 'SOL')
          
        # return the whole dataset as 1 list of tokens
        total_token_list = []
        for row in range(train_df.shape[0]):
          total_token_list.extend(train_df.loc[row, 'tokenized_SMILES'])      # can be combined with above.
        return total_token_list

    def __len__(self):
        return len(self.words_indexes) - self.args.sequence_length   # !!!!

    def __getitem__(self, index):

        return (
            torch.tensor(self.words_indexes[index:index+self.args.sequence_length]),  # input      # turn into cuda tensor?
            torch.tensor(self.words_indexes[index+1:index+self.args.sequence_length+1]),    # label
        )

##Standard DS Testing

0 = 'UNK'\
1 = 'SOL'\
2 = 'EOL'\

Es sollte als jeder Batch mit 1 beginnen, und mit 2 enden.

In [ ]:
dataset = Dataset(args)
subset_indices = [0]

subset = torch.utils.data.Subset(dataset, subset_indices)

#testloader_subset = torch.utils.data.DataLoader(subset, batch_size=args.batch_size, num_workers=0, shuffle=False)
testloader_subset = DataLoader(subset, batch_size=args.batch_size, shuffle=False)

for batch, (x, y) in enumerate(testloader_subset):
            print('\nbatch: ', batch, '\n(x, y):', (x, y))

In [ ]:
#dataset.args.batch_size

In [ ]:
#for batch, (x, y) in enumerate(dataloader):
#            print('\nbatch: ', batch, '\n(x, y):', (x, y))

##Massive Dataset class RUN

In [7]:
class Massive_Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        args,
    ):
        #self.args = args
        self.offsets = dict_offset  #offset_meta
        self.length_of_dataset = 14 #dataset_length
        self.data_path = '/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/CID-SMILES-first100'
        # should be reset in __iter__
        self.data_stream = open(self.data_path, 'r')
        self.current_offset = 0

        self.uniq_words = ['UNK', 'SOL', 'EOL', '(', ')', 'O', 'S', '1', '#', '6', '=', 'C', 'N', '7', '[N+]', '2', '4', 'P', '[O-]', '3', '5', 'Cl']  # vocab for the first 100 of CID_SMILES!
        # tokenization dictionaries (numerization)
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

    def __len__(self):
        return self.length_of_dataset

    def line_to_instance(self, line):
        tokenized_SMILES = atomwise_tokenizer(line)
        tokenized_SMILES.insert(0, 'SOL')
        tokenized_SMILES.append('EOL')
        tokenized_SMILES = [self.word_to_index[w] if (w in self.uniq_words) else self.word_to_index['UNK'] for w in tokenized_SMILES]
        return tokenized_SMILES

    def __getitem__(self, line):
        offset = self.offsets[line]        
        self.data_stream.seek(offset)
        line = self.data_stream.readline()
        line = line.rstrip("\n")
        match = re.match(pattern = "^\d+\\t(.+)$", string = line)
        if match:
          line = match.group(1)
        else:
          raise ValueError('I couldn\'t find a SMILES in a line of the dataset. :(')

        instance = self.line_to_instance(line)
        label_instance = instance[1:]
        label_instance.append(0)    # Currently, the last token is a 0. It is
                                    # debatable whether that is suitable, but
                                    # I slice it out later aynway, so I just
                                    # let that 0 be for now.
        
        '''
        # Padding with -1        
        instance += [-1] * (args.sequence_length - len(instance))
        label_instance += [-1] * (args.sequence_length - len(label_instance))
        '''
        
        return (                                                                                   # turn into cuda tensor?
            torch.tensor(instance),
            torch.tensor(label_instance),  # is the second tensor really used as a label tensor, or is it the other way around?
        )
        

##Massive DS testing

In [ ]:
dataset = Massive_Dataset(args)
              # offset_length = 100
              # offset_meta = dict_offset

subset_indices = [0]  # [0,1,2,3,4]

subset = torch.utils.data.Subset(dataset, subset_indices)

#testloader_subset = torch.utils.data.DataLoader(subset, batch_size=args.batch_size, num_workers=0, shuffle=False)
testloader_subset = DataLoader(subset, batch_size=args.batch_size, shuffle=False)

for batch, (x, y) in enumerate(testloader_subset):
            print('\nbatch: ', batch,
                  '\n(x, y):', (x, y))


batch:  0 
(x, y): (tensor([[ 1, 11, 11,  3, 10,  5,  4,  5, 11,  3, 11, 11,  3, 10,  5,  4,  5,  4,
         11, 14,  3, 11,  4,  3, 11,  4, 11,  2]]), tensor([[11, 11,  3, 10,  5,  4,  5, 11,  3, 11, 11,  3, 10,  5,  4,  5,  4, 11,
         14,  3, 11,  4,  3, 11,  4, 11,  2,  0]]))


#Model RUN

In [8]:
#import torch
#from torch import nn

class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3 #1

        n_vocab = len(dataset.uniq_words)       # provide predermined number here!
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2#,  #0,
            #bidirectional=True
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)  # input_size, hidden_size, num_layers
        logits = self.fc(output)

        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),    # initializatio with zeros?
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))
                

    #def get_hidden_state(self, x):
    #    out, (h_n, c_n) = self.lstm(embed, prev_state)      # problem: usually, I provide the entire dataset as an input here. That is not possible. => I need to extract this information at the end of the training.
    #    return h_n, c_n
        

#Train RUN

In [9]:
#import argparse
#import torch
#import numpy as np
#from torch import nn, optim
#from torch.utils.data import DataLoader

def train(dataset, model, args):
    model.train()       # set the model to training mode

    dataloader = DataLoader(
      dataset,
      batch_size=args.batch_size,
    )

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(args.max_epochs):
        state_h, state_c = model.init_state(args.sequence_length)   # Here, I need the sequence length for initializing the LSTM anew for this epoch?
                                                                    # Why is it always initialized with zeros? 

                                                                    # The argument here must not be args.sequence_length, but the length of the current sequence!
                                                                    # Or something else in a batch, so the length of the padded batch, or the length of the
                                                                    # concatenated sequences or so?

        #state_h, state_c = model.init_state(x.size()[1])


        for batch, (x, y) in enumerate(dataloader):

            print("Type of x: ", type(x), "\n")
            print("x: ", x, "\n")
            print("dims of x: ", x.size(), "\n")

            optimizer.zero_grad()

            '''
            S1/ this is a sequence
            S2/ a sequence

            [
            [-1, 0, 1, 2, 3, -2], [0, 1, 2, 3, -2, -1][/2]
            [-1, 2, 3, -2, None, None], [2, 3, -2, -1, None, None][/2]
            ]

            -- in training this is
            [-1, 0, 1, 2, 3, -2, -1, 2, 3, -2,] [0, 1, 2, 3, -2, -1, 2, 3, -2, -1,]

            [-1, 0, 1, 2, 3, -2, -1, -1, 2, 3, -2,] [0, 1, 2, 3, -2, -1, -1, 2, 3, -2, -1,]
            '''
            # here: slice x and y, leave out the last token (don't predict stuff after)

            # I padded with -1
            # Attention: This slicing needs to be adapted for batch size >1!
            '''
            numpy_version = x.numpy()
            padding_indices = np.where(numpy_version == -1)
            while -1 in numpy_version:
              numpy_version = numpy_version[:(padding_indices[0][0]-1)]
            x = torch.tensor(numpy_version)

            numpy_version = y.numpy()
            padding_indices = np.where(numpy_version == -1)
            while -1 in numpy_version:
              numpy_version = numpy_version[:(padding_indices[0][0]-1)]
            y = torch.tensor(numpy_version)
            '''


            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })
    

def predict(dataset, model, text, next_words=100):   # use this function to output the states at all timesteps!
    words = text.split(' ')
    model.eval()

    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])
        
    return words

#Apply RUN

In [10]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--max-epochs', type=int, default=10)
parser.add_argument('--batch-size', type=int, default=1)
parser.add_argument('--sequence-length', type=int, default=300) # pad with Nones, which i slice out later.
args = parser.parse_args("")

Change the dataset in dataset.py.\
Set max epochs, batch size, seqence length in train.py.

In [11]:
#dataset = Dataset(args)
dataset = Massive_Dataset(args)

model = Model(dataset)   # to cuda

In [12]:
train(dataset, model, args)

Type of x:  <class 'torch.Tensor'> 

x:  tensor([[ 1, 11, 11,  3, 10,  5,  4,  5, 11,  3, 11, 11,  3, 10,  5,  4,  5,  4,
         11, 14,  3, 11,  4,  3, 11,  4, 11,  2]]) 

dims of x:  torch.Size([1, 28]) 



RuntimeError: ignored

https://stackoverflow.com/questions/57148617/runtimeerror-expected-hidden-size-2-24-50-got-2-30-50


In [22]:
print(predict(dataset, model, text='C C ( O 2 ) C'))

['C', 'C', '(', 'O', '2', ')', 'C', '(', '=', 'C', '1', ')', 'O', ')', 'C', '=', 'O', 'EOL', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'SOL', 'C', 'C', '(', '=', 'C', 'C', '(', '=', 'C', '1', ')', 'O', ')', 'C', '=', 'O', 'EOL', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'SOL', 'C', 'C', 'C', '(', '=', 'C', 'C', '(', '=', 'C', '1', ')', 'O', ')', 'C', '=', 'O', 'EOL', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK']


# Visualization

In [ ]:
!nvidia-smi 